# First 1D Water Isotope Diffusion Model

Liam Kirkpatrick

Jan 17, 2024

IBIS project on water isotope diffusion

### Import packages

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math

### Define Forward Euler scheme algorithm for solving diffusion equation

Here I draw from: https://hplgit.github.io/fdm-book/doc/pub/diffu/pdf/diffu-4print.pdf and https://zsylvester.github.io/post/diffusion_equation/. However, I end up choosing to write my own equation


In [ ]:
def diffusion(t,z,z0,zmax,I,D):
    
    # find dt and dz
    dt = t[1]-t[0]
    dz = z[1]-z[0]
    
    # find number of timesteps and distance steps
    nt = len(t)
    nz = len(z)
    
    # calculate diffusion number
    d = D * dt / (dz**2)
    
    # create matrix of results. Set full of zeros for now
    T = np.zeros(nt,nz)
    
    # set first line of matrix to initial conditions
    T[0,:] = I
    T[0,0] = z0
    T[0,-1] = zmax
    
    # loop through all timesteps (from 1:end)
    for j in range(1,nt):
        
        # loop through all grid points (except left/right side)
        for i in range(1,nz-1)
            
            # solve for gridpoint at this time step
                # this draws on this point, the point above, and the point below at the prior timestep
            T[j,i] = T[j-1,i] + d*(T[j-1,i-1] - 2*T[j-1,i] + T[j-1,i+1])

        # set endpoints
        T[i,0] = z0
        T[i,-1] = zmax
        
    return(T)

### Define initial and boundary conditions, time vector, etc


### Run at a signle temperature

In [2]:
# set temperature (in Kelvin)
T = 250

# Calculate diffusion coefficent (following Gkinis et al., 2014)
D = 9.2 * 10**(-4) * math.exp(-7186/T) # units m^2/s



### Plot Result

In [1]:
nt = 5
t = [0,1,2,3,4]
for i in range(1,nt):
    print(i)

1
2
3
4


In [3]:
t[-1]

4